# Скачиваем французские богослужебные тексты

**Задача:** обкачать [кусочек форума forum-orthodoxe.com](http://www.forum-orthodoxe.com/~forum/viewforum.php?f=8) и вытащить оттуда содержимое всех тем подраздела.

In [1]:
from bs4 import BeautifulSoup
from lxml import html
# tqdm нужен для визуализации прогресса, если его нет, это нестрашно
from tqdm import tqdm
from urllib import request

import re
import time

## Базовые функции

In [14]:
source_pages = ["http://www.forum-orthodoxe.com/~forum/viewforum.php?f=8",
               "http://www.forum-orthodoxe.com/~forum/viewforum.php?f=8&start=50"]

Сначала нужно получить содержимое страницы (любой!), чтобы с ней работать — с этим поможет _BeautifulSoup_. В дальнейшем, чтобы получить количество страниц в обсуждении, понадобятся ещё и XPath-выражения (о них понятно написано в [этой статье на Хабре](https://habr.com/ru/post/114772/), в качестве бонуса — скрины Windows XP из 2011), а с ними удобнее работать через _lxml,_ поэтому его тоже будем отдавать, но только если попросят (`needs_lxml=True`).

In [43]:
def parse_page(page_url, needs_lxml=False):
    """Скачивает всё содержимое страницы.
    
    :arg page_url — (str) страница, которую нужно обработать
    :arg needs_lxml — (bool, необязательный) нужен ли LMXL-вариант для работы с XPath-
    выражениями
    
    :returns soup — (bs4.BeautifulSoup) готовое к парсингу содержимое
    страницы
    :returns page_tree — (lxml.html) готовое к работе содержимое страницы
    для работы с xpath-выражениями
    """
    user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"
    req = request.Request(page_url, headers={"User-Agent":user_agent})
    with request.urlopen(req) as response:
        page = response.read().decode("utf-8")
        soup = BeautifulSoup(page, "html.parser")
        if needs_lxml:
            page_tree = html.fromstring(page)
            return soup, page_tree
        else:
            return soup

In [10]:
# это нужно для того, чтобы собрать ссылки на страницы веток форума
base_url = "http://www.forum-orthodoxe.com/~forum/"

## Обрабатываем исходные страницы

Для каждой из наших двух исходных страниц нужно сделать следующее:

1) скачать страницу в двух форматах — BeautifulSoup и LXML;

2) найти там все ссылки в ветки;

3) для каждой ветки проверить — вдруг в ней больше одной страницы обсуждений? Если это так, то определить их количество по тегу `<li>` и сгенерировать ссылки на каждую страницу отдельно;

4) сохранить результат работы в словарь, где ключ — это название ветки, а значение — список ссылок на все страницы обсуждения этой ветки.

In [53]:
def get_threads_and_pages(source_page, threads_hrefs):
    """Проходится по страничкам форума с темами, собирает ссылки на 
    все страницы тем.
    
    :arg source_page — (str) страничка, которые нужно обработать
    :arg threads_hrefs — dict {str: list of str} список тем (заголовок) 
    и все страницы этой темы
    
    :returns threads_hrefs — dict {str: list of str} сдополненный список тем 
    и ссылок на страницы этой темы"""
    source_soup, source_tree = parse_page(source_page, needs_lxml=True)
    threads_raw = source_soup.find_all("a", attrs={"class": "topictitle"})
    for th_r in threads_raw:
        th_title = th_r.get_text()
        th_base_href = base_url + th_r["href"][2:]
        try:
            pages_xpath = "//a[@class='topictitle' and @href='{}']/".format(th_r["href"]) + \
                            "../div[@class='pagination']/ul/li"
            len_pages = len(source_tree.xpath(pages_xpath))
        except:
            len_pages = 1
        th_pages = [th_base_href]
        if len_pages > 1:
            for i in range(1, len_pages):
                count = 15 * i
                new_page = th_base_href + "&start={}".format(count)
                th_pages.append(new_page)
        threads_hrefs[th_title] = th_pages
    return threads_hrefs

Обкачиваем страницы с темами:

In [54]:
threads_hrefs = {}
for src_page in source_pages:
    threads_hrefs = get_threads_and_pages(src_page, threads_hrefs)

Вот что получилось:

In [60]:
for title in sorted(threads_hrefs):
    print("{}\n  - {}".format(title, "\n  - ".join(threads_hrefs[title])))

21 octobre: prière aux confesseurs de Transylvanie
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=2675&sid=d14b6f440d66fbbdf31e6b93347a8a20
25 mars Annonciation
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=1808&sid=5116e515a298362234bf135e43a884e8
28 mars saint Gontran (I)
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=1824&sid=d14b6f440d66fbbdf31e6b93347a8a20
3ème dimanche de carême
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=1802&sid=5116e515a298362234bf135e43a884e8
Acathiste à la Mère de Dieu
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=1804&sid=5116e515a298362234bf135e43a884e8
Acathiste à saint Etienne le Grand de Moldavie
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=2630&sid=d14b6f440d66fbbdf31e6b93347a8a20
Acathistes et Canons
  - http://www.forum-orthodoxe.com/~forum/viewtopic.php?f=8&t=1806&sid=d14b6f440d66fbbdf31e6b93347a8a20
Dimanche de la Samaritaine
  - http://www.forum-orthod

Сохраняем результаты работы:

In [ ]:
import json

In [61]:
with open("./textes_orthodoxes_fr/links.json", "w", encoding="utf-8") as backup:
    json.dump(threads_hrefs, backup)

## Вытаскиваем тексты

Для каждой ветки, которая у нас лежит в `threads_hrefs`, нужно:

1) скачать только BeautifulSoup версию страницы (`needs_lxml=False`);

2) вычленить только те куски HTML-кода, в которых находится текст сообщений, чтобы не собрать лишнее;

3) из этих кусков вытащить текст (`.get_text()`): один HTML-кусок = одно сообщение — и сложить всё это в общий список сообщений;

4) сообщения собрать в одну строку с каким-нибудь разделителем (например, `\n===\n`);

5) сохранить получившийся результат в файл `.txt` с названием обработанной ветки.

In [64]:
for title in tqdm(threads_hrefs):
    thread_pages = threads_hrefs[title]
    parsed_texts = []
    for page in thread_pages:
        page_soup = parse_page(page)
        for i, item in enumerate(page_soup.findAll("div",attrs={"class":"content"})):
            parsed = re.sub("\n{2,}", "\n", item.get_text())
            parsed_texts.append(parsed)
    full_post_text = "\n===\n".join(parsed_texts)
    with open("./textes_orthodoxes_fr/{}.txt".format(title), "w", encoding="utf-8") as f:
        f.write(full_post_text)

100%|██████████| 60/60 [00:52<00:00,  1.14it/s]


Последняя версия корпуса на моём гугл-диске: [клик!](https://drive.google.com/file/d/1cdxhhk2v94nCEfLG3RcHY1USUa0R2B6T/view?usp=sharing)